In [37]:
import json
import pandas
import os
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from  itertools import tee, chain, islice
from math import log
from operator import itemgetter
from itertools import takewhile

In [38]:
root_dir = '/home/marcin/Desktop/SemestrVIII/PJN'
year = "2018"
json_data_dir = f"{root_dir}/data/json"
filesInYearPattern = 'judgments-(316[3-9]|317\d)\.json'

In [39]:
def window(it, size=2):
    yield from zip(*[islice(it, s, None) for s, it in enumerate(tee(it, size))])

In [40]:
def unigrams(line):
    notags = re.sub(r"<[^>]*>", " ", line)
    nobreaks =  re.sub(r"-\n", " ", notags)
    nodigits =  re.sub(r"\d+", " ", nobreaks)
    noroman =  re.sub(r"\b[XVILMC]+\b", "", nodigits)
    lower =   noroman.lower()
    return  re.findall(r'\w+', lower)

In [41]:
def make_bigrams(unigrams):
    bigram_tuples = window(unigrams,2)
    bigrams = map(lambda tuple: f"{tuple[0]} {tuple[1]}", bigram_tuples) 
    return bigrams

In [42]:
def judgement_texts(filename):
    with open(os.path.join(json_data_dir, filename), 'r') as jsonFile:
        judgements = json.load(jsonFile)['items'] 
    year_filtered = filter(lambda item: year in item['judgmentDate'], judgements)
    yield from map(lambda item: item['textContent'], year_filtered)

In [43]:
def make_bigrams(unigrams):
    bigram_tuples = window(unigrams,2)
    bigrams = map(lambda tuple: f"{tuple[0]} {tuple[1]}", bigram_tuples) 
    return bigrams

In [44]:
json_files = os.listdir(json_data_dir)
judgements_files= filter(lambda name: re.match(filesInYearPattern, name), json_files)
texts =  chain.from_iterable(map( judgement_texts, judgements_files))
unigrams_it1, unigrams_it2 =tee(map(unigrams,texts))
unigrams =  chain.from_iterable(unigrams_it1)
bigrams = chain.from_iterable(map( lambda it: window(it,2),unigrams_it2))
   

In [45]:
counted_unigrams= Counter(unigrams)
counted_bigrams= Counter(bigrams)

unigram_count = sum(counted_unigrams.values())
bigram_count =  sum(counted_bigrams.values())

# 10 najpopularniejszych bigramów w 2018

In [46]:
counted_bigrams.most_common(10)

[(('z', 'dnia'), 11869),
 (('art', 'k'), 7184),
 (('w', 'dniu'), 5208),
 (('na', 'podstawie'), 4699),
 (('k', 'p'), 4686),
 (('w', 'sprawie'), 4657),
 (('z', 'art'), 4453),
 (('w', 'tym'), 4196),
 (('art', 'ust'), 3575),
 (('zgodnie', 'z'), 3378)]

# 10 najpopularniejszych unigramów w 2018

In [47]:
counted_unigrams.most_common(10) 

[('w', 118896),
 ('z', 69578),
 ('na', 42894),
 ('i', 39428),
 ('do', 37881),
 ('nie', 30528),
 ('o', 29826),
 ('k', 28916),
 ('r', 27356),
 ('że', 22479)]

In [48]:
def pmi(bigram):
    prob_word_1 = counted_unigrams[bigram[0]] / unigram_count
    prob_word_2 = counted_unigrams[bigram[1]] / unigram_count
    prob_bigram = counted_bigrams[bigram] /     bigram_count 
    return log( prob_bigram / (prob_word_1 * prob_word_2))    

In [49]:
def top_pmi(min_frequency = 1):
    frequent_bigrams = takewhile(lambda x: x[1] > min_frequency, counted_bigrams.most_common() )
    pmi_vals = map(lambda x: (x[0],pmi(x[0])),frequent_bigrams)
    return list(sorted(pmi_vals,key = itemgetter(1),reverse=True))

# top 30 bigrams by PMI

In [50]:
top_pmi(100)[:30]

[(('samorządowej', 'jednostce'), 9.594232074185577),
 (('trybunału', 'konstytucyjnego'), 9.589584116568494),
 (('zdrowotnej', 'finansowanych'), 9.518455006864993),
 (('księgę', 'wieczystą'), 9.517003377571543),
 (('sfery', 'budżetowej'), 9.410511276635498),
 (('doznaną', 'krzywdę'), 9.405390343875302),
 (('uzasadnieniem', 'doręczyć'), 9.36187456907651),
 (('stawek', 'dziennych'), 9.240075327075525),
 (('grach', 'hazardowych'), 9.22696819871096),
 (('gospodarstwa', 'rolnego'), 9.194750897930989),
 (('jednostce', 'sfery'), 9.122246283735851),
 (('obszaru', 'ograniczonego'), 9.098896619410258),
 (('gospodarstwie', 'rolnym'), 9.026640589888276),
 (('ograniczonego', 'użytkowania'), 8.980475404382185),
 (('służby', 'wojskowej'), 8.964687070754378),
 (('szczecin', 'centrum'), 8.927170559858226),
 (('godzinach', 'nadliczbowych'), 8.923842599384564),
 (('tekst', 'jednolity'), 8.918322125917946),
 (('utraciła', 'zdolność'), 8.858359840264844),
 (('doświadczenia', 'życiowego'), 8.836677207577926)

In [51]:
top_pmi(1)[:30]

[(('ogórków', 'rozpaletyzowanych'), 13.985007455438776),
 (('delikatesów', 'mięsnych'), 13.985007455438776),
 (('czerwona', 'kropka'), 13.985007455438776),
 (('podłogach', 'grzyb'), 13.985007455438776),
 (('poplamione', 'pozaciągane'), 13.985007455438776),
 (('pozaciągane', 'wymięte'), 13.985007455438776),
 (('odczyn', 'opłucnowy'), 13.985007455438776),
 (('zaspokajających', 'najpotrzebniejsze'), 13.985007455438776),
 (('kantak', 'rekowska'), 13.985007455438776),
 (('zasypowych', 'zadaszeń'), 13.985007455438776),
 (('przenośnikiem', 'taśmowym'), 13.985007455438776),
 (('geologiczno', 'geotechnicznymi'), 13.985007455438776),
 (('ulotkami', 'reklamowymi'), 13.985007455438776),
 (('catering', 'gesellschaftmbh'), 13.985007455438776),
 (('jakiemuś', 'sezonowi'), 13.985007455438776),
 (('taneczno', 'rytmicznych'), 13.985007455438776),
 (('oferujące', 'trampoliny'), 13.985007455438776),
 (('bon', 'aktywacyjny'), 13.985007455438776),
 (('odblokowania', 'immobilizera'), 13.985007455438776),
 ((

In [52]:
from  python_llr.llr import llr_2x2
def loglikelihood(bigram):
   k11 = counted_bigrams[bigram] 
   k12 = counted_unigrams[bigram[0]] - counted_bigrams[bigram]
   k21 = counted_unigrams[bigram[0]] - counted_bigrams[bigram]
   k22 = bigram_count - (counted_unigrams[bigram[0]] + counted_unigrams[bigram[1]] - counted_bigrams[bigram])
   return llr_2x2(k11,k12,k21,k22) 

# top 30 bigrams by loglikelihood

In [53]:
log_like_vals = map(lambda bigram: (bigram, loglikelihood(bigram)) , counted_bigrams.keys())
list(sorted(log_like_vals,key = itemgetter(1),reverse=True))[:30]

[(('art', 'k'), 48150.98838423297),
 (('zgodnie', 'z'), 46090.114974131175),
 (('ubezpieczeń', 'społecznych'), 38436.001355060034),
 (('sygn', 'akt'), 34531.62656993238),
 (('zw', 'z'), 32088.484780825333),
 (('związku', 'z'), 29483.45356562183),
 (('stycznia', 'r'), 27455.6964131868),
 (('z', 'dnia'), 25596.18387534353),
 (('dz', 'u'), 24742.67530888688),
 (('p', 'c'), 23040.262288962957),
 (('sp', 'z'), 22930.5772499504),
 (('grudnia', 'r'), 21317.997934416053),
 (('października', 'r'), 20641.980759875914),
 (('wraz', 'z'), 20216.365461446927),
 (('podstawie', 'art'), 20160.379198322524),
 (('uwagi', 'na'), 20101.871001320647),
 (('kwotę', 'zł'), 18385.62557430438),
 (('organ', 'rentowy'), 18330.33853901432),
 (('lipca', 'r'), 18060.943811455327),
 (('ocenie', 'sądu'), 18011.78364027379),
 (('listopada', 'r'), 17892.50105247245),
 (('art', 'ust'), 17443.330567661615),
 (('września', 'r'), 17085.267792186656),
 (('k', 'p'), 17030.818500855006),
 (('niezdolności', 'do'), 16749.43888060

In [54]:
log_like_vals_weighted = map(lambda bigram: (bigram, log(counted_bigrams[bigram]) * loglikelihood(bigram)) , counted_bigrams.keys())
list(sorted(log_like_vals_weighted,key = itemgetter(1),reverse=True))[:30]

[(('art', 'k'), 427562.07548874523),
 (('zgodnie', 'z'), 374483.98616699036),
 (('ubezpieczeń', 'społecznych'), 302556.38991255977),
 (('sygn', 'akt'), 269955.3224631211),
 (('zw', 'z'), 244662.47029696775),
 (('z', 'dnia'), 240135.34041389034),
 (('związku', 'z'), 226178.21356401715),
 (('stycznia', 'r'), 213464.07958588324),
 (('p', 'c'), 185350.29950252373),
 (('dz', 'u'), 181535.44781765417),
 (('sp', 'z'), 166487.97431050992),
 (('grudnia', 'r'), 159479.7754754352),
 (('podstawie', 'art'), 154191.88291149423),
 (('października', 'r'), 153163.08282289314),
 (('wraz', 'z'), 144449.89635882087),
 (('k', 'p'), 143950.17680583615),
 (('uwagi', 'na'), 143930.7875716255),
 (('art', 'ust'), 142716.45451099842),
 (('kwotę', 'zł'), 134870.21974185703),
 (('lipca', 'r'), 131182.58810532847),
 (('ocenie', 'sądu'), 130143.80898456149),
 (('organ', 'rentowy'), 130054.93199177393),
 (('listopada', 'r'), 129971.66697702144),
 (('co', 'do'), 125070.89081724465),
 (('września', 'r'), 123046.6689019